In [1]:
# импортируем необходимые библиотеки pandas и numpy,
# функцию train_test_split() и классы StandardScaler, 
# OneHotEncoder, TransformerMixin, 
# LogisticRegression, Pipeline
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import (StandardScaler, 
                                   OneHotEncoder)
from sklearn.base import TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [2]:
# записываем CSV-файл в объект DataFrame
data = pd.read_csv('Data/StateFarm_missing.csv', sep=';')

In [3]:
# разбиваем данные на обучающие и тестовые: 
# получаем обучающий массив признаков, 
# тестовый массив признаков, обучающий
# массив меток, тестовый массив меток
X_train, X_test, y_train, y_test = train_test_split(
    data.drop('Response', axis=1), 
    data['Response'], 
    test_size=0.3,
    stratify=data['Response'],
    random_state=42)

In [4]:
# создаем класс, выполняющий импутацию
class DFImputer(TransformerMixin):
    def __init__(self):
        self.fill = None
        
    def fit(self, X, y=None):
        # вычисляем моды (берем первую) для 
        # категориальных переменных
        # и медианы - для количественных
        self.fill = pd.Series([X[c].mode()[0]
            if X[c].dtype == np.dtype('O') else X[c].median() 
                               for c in X], index=X.columns)
        return self
    
    def transform(self, X, y=None):
        # заменяем модами пропуски в категориальных переменных
        # и медианами - пропуски в количественных переменных
        Xfill = X.fillna(self.fill)
        return Xfill

In [5]:
# создаем класс, выполняющий стандартизацию
class DFStandardScaler(TransformerMixin):

    def __init__(self):
        pass

    def fit(self, X, y=None):
        self.ss = StandardScaler()
        # выделяем список количественных переменных
        num_cols = X.select_dtypes(
            exclude='object').columns.tolist()
        # вычисляем средние (mean_) и стандартные отклонения (scale_)
        # для количественных переменных
        self.ss.fit(X[num_cols])
        return self

    def transform(self, X):
        # выделяем список количественных переменных
        num_cols = X.select_dtypes(
            exclude='object').columns.tolist()
        # выполняем стандартизацию количественных переменных
        X[num_cols] = self.ss.transform(X[num_cols])
        # преобразовываем массив NumPy в объект DataFrame
        X_scaled = pd.DataFrame(X, index=X.index, columns=X.columns)
        return X_scaled

In [6]:
# создаем класс, выполняющий дамми-кодирование
class DFOneHotEncoder(TransformerMixin):

    def __init__(self):
        pass

    def fit(self, X, y=None):
        # выделяем список категориальных переменных
        cat_cols = X.select_dtypes(
            include='object').columns.tolist()
        # определяем дамми-переменные для 
        # категориальных переменных
        self.ohe = OneHotEncoder(sparse=False, 
                                 handle_unknown='ignore')
        self.ohe.fit(X[cat_cols])
        return self

    def transform(self, X):
        # выделяем списки количественных 
        # и категориальных переменных
        num_cols = X.select_dtypes(
            exclude='object').columns.tolist()
        cat_cols = X.select_dtypes(
            include='object').columns.tolist()
        # выполняем дамми-кодирование 
        # категориальных переменных
        Xt = self.ohe.transform(X[cat_cols])
        # извлекаем имена дамми-переменных
        cols = self.ohe.get_feature_names_out()
        # превращаем массив NumPy с дамми-переменными
        # в объект DataFrame
        X_dum = pd.DataFrame(Xt, index=X[cat_cols].index, 
                             columns=cols)
        # конкатерируем датафрейм, содержащий 
        # количественные переменные, с датафреймом, 
        # содержащим дамми-переменные
        X_res = pd.concat([X[num_cols], X_dum], axis=1)
        return X_res

In [7]:
# загружаем игрушечные наборы для тестирования классов
toy_train = pd.read_csv('Data/toy_train.csv', sep=';')
toy_test = pd.read_csv('Data/toy_test.csv', sep=';')

In [8]:
# смотрим игрушечный обучающий набор
toy_train

,age,income,region
0,NaN,NaN,MSK
1,23.0,4560.55,MSK
2,24.0,NaN,NaN
3,30.0,NaN,EKAT
4,NaN,7888.10,NaN
5,55.0,9000.50,SPB
6,37.0,NaN,SPB


In [9]:
# смотрим игрушечный тестовый набор
toy_test

,age,income,region
0,89.0,903.33,MSK
1,23.0,4560.55,NSK
2,24.0,NaN,MSK
3,55.0,6700.00,MSK
4,NaN,8999.00,EKAT
5,NaN,5430.00,SPB
6,37.0,NaN,NaN


In [10]:
# вычислим медианы и моды
print(toy_train['age'].median())
print(toy_train['income'].median())
print(toy_train['region'].mode())

30.0
7888.1
0    MSK
1    SPB
dtype: object


In [11]:
# тестируем класс, выполняющий импутацию,
# на игрушечных данных
imp = DFImputer()
imp.fit(toy_train)
toy_train = imp.transform(toy_train)
toy_test = imp.transform(toy_test)

In [12]:
# смотрим игрушечный обучающий набор
# после импутации
toy_train

,age,income,region
0,30.0,7888.10,MSK
1,23.0,4560.55,MSK
2,24.0,7888.10,MSK
3,30.0,7888.10,EKAT
4,30.0,7888.10,MSK
5,55.0,9000.50,SPB
6,37.0,7888.10,SPB


In [13]:
# смотрим игрушечный тестовый набор
# после импутации
toy_test

,age,income,region
0,89.0,903.33,MSK
1,23.0,4560.55,NSK
2,24.0,7888.10,MSK
3,55.0,6700.00,MSK
4,30.0,8999.00,EKAT
5,30.0,5430.00,SPB
6,37.0,7888.10,MSK


In [14]:
# тестируем класс, выполняющий стандартизацию,
# на игрушечных данных
scaler = DFStandardScaler()
scaler.fit(toy_train)
toy_train = scaler.transform(toy_train)
toy_test = scaler.transform(toy_test)

In [15]:
# смотрим игрушечный обучающий набор
# после стандартизации
toy_train

,age,income,region
0,-0.270000,0.245729,MSK
1,-0.966315,-2.338170,MSK
2,-0.866841,0.245729,MSK
3,-0.270000,0.245729,EKAT
4,-0.270000,0.245729,MSK
5,2.216840,1.109526,SPB
6,0.426315,0.245729,SPB


In [16]:
# смотрим игрушечный тестовый набор
# после стандартизации
toy_test

,age,income,region
0,5.598941,-5.178063,MSK
1,-0.966315,-2.338170,NSK
2,-0.866841,0.245729,MSK
3,2.216840,-0.676851,MSK
4,-0.270000,1.108361,EKAT
5,-0.270000,-1.663027,SPB
6,0.426315,0.245729,MSK


In [17]:
# тестируем класс, выполняющий дамми-кодирование,
# на игрушечных данных
ohe = DFOneHotEncoder()
ohe.fit(toy_train)
toy_train = ohe.transform(toy_train)
toy_test = ohe.transform(toy_test)

In [18]:
# смотрим игрушечный обучающий набор
# после дамми-кодирования
toy_train

,age,income,region_EKAT,region_MSK,region_SPB
0,-0.270000,0.245729,0.0,1.0,0.0
1,-0.966315,-2.338170,0.0,1.0,0.0
2,-0.866841,0.245729,0.0,1.0,0.0
3,-0.270000,0.245729,1.0,0.0,0.0
4,-0.270000,0.245729,0.0,1.0,0.0
5,2.216840,1.109526,0.0,0.0,1.0
6,0.426315,0.245729,0.0,0.0,1.0


In [19]:
# смотрим игрушечный тестовый набор
# после дамми-кодирования
toy_test

,age,income,region_EKAT,region_MSK,region_SPB
0,5.598941,-5.178063,0.0,1.0,0.0
1,-0.966315,-2.338170,0.0,0.0,0.0
2,-0.866841,0.245729,0.0,1.0,0.0
3,2.216840,-0.676851,0.0,1.0,0.0
4,-0.270000,1.108361,1.0,0.0,0.0
5,-0.270000,-1.663027,0.0,0.0,1.0
6,0.426315,0.245729,0.0,1.0,0.0


In [20]:
# создаем конвейер
ml_pipe = Pipeline([
    ('impute', DFImputer()), 
    ('scaler', DFStandardScaler()),
    ('ohe', DFOneHotEncoder()), 
    ('logreg', LogisticRegression(solver='lbfgs', 
                                  max_iter=200))
])

In [21]:
# обучаем итоговый конвейер
ml_pipe.fit(X_train, y_train)
# оцениваем качество модели на обучающих данных
print('Правильность на обучающей выборке: {:.3f}'.format(
    ml_pipe.score(X_train, y_train)))
# оцениваем качество модели на тестовых данных
print('Правильность на тестовой выборке: {:.3f}'.format(
    ml_pipe.score(X_test, y_test)))

Правильность на обучающей выборке: 0.900
Правильность на тестовой выборке: 0.899
